### Objective

A Comparison between Statistical Classification Models' ability to learn and identify Human vs. Robot based on movement patterns and trajectory.

### The Data

Annotated dataset for multi-camera tracking and 2D/3D object detection.

- **Data Source:**  
  [NVIDIA PhysicalAI SmartSpaces Dataset (Hugging Face)](https://huggingface.co/datasets/nvidia/PhysicalAI-SmartSpaces)

- **Data Description:**  
  - Consists of videos from indoor scenes (warehouses, hospitals, retail spaces).
  - Time-synchronized to track humans and robots across multiple cameras.
  - For this project, we use the **calibration** and **ground truth annotation** files from a 5-minute simulation.
  - Each frame captures:
    - Bounding boxes of each object in each camera view.
    - Pixel area (feature) and Euclidean distance from the origin (target).
  - The training dataset contains **14 different warehouses**, each with:
    - A varying number of cameras.
    - A varying number of moving units (Humans/Robots).

### Predictors and Response
The dataset's starting point is the tracked movement of a unit within the space as captured by the sensors. 
The data processing invloves collecting statistics about the unit's movement.

- Input Features: 
  First and second order statistics of a unit's movement and trajectory within the physical space as captured by the SmarSpace sensors, including:
  * minimal distance from origin (center of map).
  * Maximal distance from origin.
  * Range of the trajectory.
  * Total distance traveled.
  * Proportion of the sensors capturing the movement; hinting at the unit's position relative to overall area coverage.

- Target Variable: 
  **Human vs. Robot**

### Models
  - Logistic Regression
  - Discriminant Analysis
  - Support Vector Machine
  - Random Forest
  - Gradient Booster

### Performance Metrics
All models are compared and benchmarked against each other on the following metrics:
  - **Balanced Accuracy** - Measuring overall correctness.

  - **True Positive Rate (Sensitivity/Recall)** - Measuring the ability to detect the positives (Robots).
  - **True Negative Rate (Specificity)** - Measuring the ability to detect the negatives (Humans).
  - **Positive Predictive Value (Precision)** - Measuring how many predicted positives are true.
  - **Negative Predictive Value** - Measuring how many predicted negatives are true.
  - **F1 Score** - Measuring a balance between precision and recall.
  - **Area Under the ROC Curve (AUC)** - Measuring a threshold-independent model performance.

In [ ]:
# Data Fetching
import numpy as np
import pandas as pd
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')
from src.config import PROCESSED_DATA_DIR
from src.classification_utils import *
from src.utils import plotly_to_confluence

# ---------------------------------------------------------------
csv_path = PROCESSED_DATA_DIR / "all_units_statistics.csv" 
# ---------------------------------------------------------------

df = pd.read_csv(csv_path)
df['UNIT_TYPE'] = df['UNIT_TYPE'].str.replace('Forklift','Robot')\
                                            .replace('NovaCarter','Robot')\
                                            .replace('Transporter','Robot')\
                                            .replace('AgilityDigit','Robot')\
                                            .replace('FourierGR1T2','Robot')

relevant_columns = [# 'MIN_LOC_X',                        
                    # 'MIN_LOC_Y',                        
                    'MIN_DIST_ORIGIN',                  
                    # 'MAX_LOC_X',                        
                    # 'MAX_LOC_Y',                        
                    'MAX_DIST_ORIGIN',                  
                    # 'MEAN_LOC_X',                       
                    # 'MEAN_LOC_Y',                       
                    # 'MEAN_DIST_ORIGIN',                 
                    # 'STD_LOC_X',                        
                    # 'STD_LOC_Y',                        
                    # 'STD_DIST_ORIGIN',                  
                    # 'MEDIAN_LOC_X',                     
                    # 'MEDIAN_LOC_Y',                     
                    # 'MEDIAN_DIST_ORIGIN',               
                    # 'RANGE_LOC_X',                      
                    # 'RANGE_LOC_Y',                      
                    'RANGE_DIST_ORIGIN',                
                    # 'TOTAL_DIST_TRAVELED_X',            
                    # 'TOTAL_DIST_TRAVELED_Y',            
                    'TOTAL_DIST_TRAVELED_EUCLIDEAN',    
                    # 'TOTAL_DIRECTION_CHANGES',          
                    # 'TIME_SPENT_WALKING_SEC',           
                    # 'TIME_SPENT_STANDING_SEC',          
                    'RATIO_OF_CAMERA_CAPTURES',         
                    # 'UNIT_ID',                                              
                    # 'WAREHOUSE', 
                    'UNIT_TYPE']

df = df[relevant_columns]
print('DataFrame Head:')
display(df.head(10))
print(f'DataFrame Shape: {df.shape}')
print(f'DataFrame description:')
display(df.describe())
print('Missing values count:')
display(df.isna().sum())
print('Targets value counts:')
display(df.UNIT_TYPE.value_counts()) # This is the binary target


In [ ]:
# Stratified Train Test Split + Performance Tuning Params
from sklearn.model_selection import train_test_split

# ------------------------------------------------------------------

random_state = 42
train_test_split_ratio = 0.2

# ------------------------------------------------------------------

X = df.drop(columns=['UNIT_TYPE'])
y = (df['UNIT_TYPE'] == 'Robot').astype(int)  # Binary target: Robot=1, Person=0

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=train_test_split_ratio, 
                                                    stratify=y, random_state=random_state)

df = df.iloc[X_train.shape[0]*[True] + X_test.shape[0]*[False]].reset_index(drop=True)

In [ ]:
# Data vizualization
import seaborn as sns
import matplotlib.pyplot as plt

sns.pairplot(df, hue="UNIT_TYPE", diag_kind="kde", height=2, aspect=1)
plt.tight_layout()
plt.show()

fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12, 7))
axes = axes.flatten()
numeric_columns = [col for col in df.columns if col != 'UNIT_TYPE']

for i, col in enumerate(numeric_columns):
    sns.boxplot(x='UNIT_TYPE', y=col, data=df, ax=axes[i])
    axes[i].set_title(f'{col}')

for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

features = df[[col for col in relevant_columns if 'UNIT_TYPE' not in col]]
labels = df['UNIT_TYPE']
label_names = np.unique(labels)
label_to_int = {name: i for i, name in enumerate(label_names)}
int_labels = np.array([label_to_int[label] for label in labels])

plt.figure(figsize=(10, 6))
sns.heatmap(pd.DataFrame(features, columns=features.columns).corr(), annot=True, cmap='coolwarm')
plt.title('Feature Correlation Heatmap')
plt.xticks(rotation=45, fontsize=8) 
plt.yticks(fontsize=8)      
plt.tight_layout()
plt.show()


### Classification Metrics used
- **TP (True Positive)**: Correctly predicted positive cases  
- **TN (True Negative)**: Correctly predicted negative cases  
- **FP (False Positive)**: Incorrectly predicted positive cases (Type I error)  
- **FN (False Negative)**: Incorrectly predicted negative cases (Type II error)

| Metric                        | Formula                                                               |
|------------------------------|----------------------------------------------------------------------------------------|
| **Balanced Accuracy**        | $(\text{Sensitivity} + \text{Specificity}) / 2$                                       |
| **Sensitivity (Recall, TPR)**| $TP / (TP + FN)$                                                                      |
| **Specificity (TNR)**        | $TN / (TN + FP)$                                                                      |
| **Precision (PPV)**          | $TP / (TP + FP)$                                                                      |
| **Negative Predictive Value (NPV)** | $TN / (TN + FN)$                                                              |
| **F1 Score**                 | $2 \cdot \frac{\text{Precision} \cdot \text{Sensitivity}}{\text{Precision} + \text{Sensitivity}}$ |
| **AUC**                      | Measures the model’s ability to distinguish between classes across all classification thresholds.                |


### Logistic Regression

Logistic regression models the probability of a binary outcome $ Y \in \{0, 1\} $ as a function of predictors $ X \in \mathbb{R}^p $.

The model:

$$
\log\left(\frac{\mathbb{P}(Y=1 \mid X)}{\mathbb{P}(Y=0 \mid X)}\right) = \beta_0 + \beta_1 x_1 + \dots + \beta_p x_p
$$

- The left-hand side is the **logit**.
- Estimation is done via **maximum likelihood**.
- Output is a probability of class.

The model assumes:
- Assumes linearity in the log-odds.
- No multicollinearity between predictors.
- Independent observations.

The **Box-Tidwell Test** checks the linearity of continuous predictors with the log-odds in logistic regression model.

For each predictor $x$, the test adds a term $x \cdot log(x)$ to the model:

$$
log (\frac{p}{1-p}) = \beta_0 + \beta_1 \cdot x + \beta_2 \cdot x\cdot log(x)
$$

$\Rightarrow$ if $\beta_2$ is not statistically significant, the relationship between log-odds and $x$ is likely linear.

In [ ]:
# A Log Odds linearity Check

import statsmodels.api as sm

y = (df['UNIT_TYPE'] == 'Robot').astype(int)
X = df.drop(columns=['UNIT_TYPE'])

# Removing non-positive values to avoid log(0) and log(negative)
X = X[(X > 0).all(axis=1)]
y = y.loc[X.index]

# Box-Tidwell transformed terms
for col in X.columns:
    X[f'{col}_log'] = X[col] * np.log(X[col])

# intercept
X = sm.add_constant(X)

# Fit logistic regression
model = sm.Logit(y, X).fit(disp=0) 

# p-values of interaction terms
pvals = model.pvalues
box_tidwell_pvals = pvals[pvals.index.str.endswith('_log')]

# Plot
minus_log_p = -np.log10(box_tidwell_pvals)
plt.figure(figsize=(10, 5))
minus_log_p.sort_values().plot(kind='barh', color='steelblue')
plt.axvline(-np.log10(0.05), color='red', linestyle='--', label='α = 0.05')
plt.xlabel('-log10(p-value)')
plt.title('Box-Tidwell Nonlinearity Test', fontsize=10)
plt.legend(loc='lower right')
plt.tight_layout()
plt.show()

* None of the bars in the $-log_{10}(\beta_2 \ p-value)$ plot pass the red $α = 0.05$ threshold line; All p-values are greater than $0.05$.
* Fail to reject the null hypothesis; No evidence of nonlinearity for any feature.

$\Rightarrow$ The Logistic Regression assumption holds for the non-correlated features.

In [ ]:
# Fitting a logistic regression model

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, GridSearchCV

# ----------------------------------------- Model Parameters  -----------------------------------------

auto_best_threshold_based_on_metric = 'accuracy'      # Available metrics: ["accuracy", "sensitivity", "specificity", "ppv", "npv", "f1", "auc"]
include_bootstrap_confidence_intervals = True        # True # False
cross_validation_splits = 5                           # 5-Fold cross validation
grid_search_scoring_metric = 'roc_auc'                 # choosing best metric in grid search : ['roc_auc','recall']

# ----------------------------------------------------------------------------------------------------

logreg = LogisticRegression(max_iter=1000, solver='liblinear', class_weight='balanced')

param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2']
}

cv = StratifiedKFold(n_splits=cross_validation_splits, shuffle=True, random_state=random_state)
grid = GridSearchCV(logreg, param_grid, cv=cv, scoring=grid_search_scoring_metric, n_jobs=-1)
grid.fit(X_train, y_train)

print("Logistic Regression Best parameters:", grid.best_params_)
# Test set evaluation
best_model = grid.best_estimator_
y_test_proba = best_model.predict_proba(X_test)[:, 1]

best_metrics_logreg = find_best_threshold(
    model=best_model,
    X_test=X_test,
    y_test=y_test,
    metric=auto_best_threshold_based_on_metric,             
    bootstrap=include_bootstrap_confidence_intervals,    # to get CI    
    n_bootstrap=100         
)

print_metrics(metrics_dict=best_metrics_logreg)

y_pred = best_model.predict(X_test)

plot_confusion_mat(best_model, 
                   X_test, y_test, 
                   threshold=best_metrics_logreg['threshold'], 
                   labels=["Human", "Robot"])

fig = plot_auc(y_test, y_test_proba)
fig.show()
# plot_auc_paper(y_test, y_test_proba)

# #------------------------ Choosing a different threshold ------------------------

# threshold_to_display = 0.5 

# threshold_dict = evaluate_model_at_threshold(best_model, 
#                             X_test, y_test, 
#                             threshold=threshold_to_display, 
#                             bootstrap=include_bootstrap_confidence_intervals, 
#                             n_bootstrap=100)

# print_metrics(metrics_dict=threshold_dict)

# plot_confusion_mat(best_model, 
#                    X_test, y_test, 
#                    threshold=threshold_dict['threshold'], 
#                    labels=["Human", "Robot"])

# #--------------------------------------------------------------------------------


### Support Vector Machine (SVM) Classifier

Support Vector Machines aim to find a hyperplane that maximally separates classes in the feature space. The separation is done by maximizing the margin between the classes, defined by the closest points (support vectors).

#### Linear SVM

Given labeled data $(\mathbf{x}_i, y_i)$, where $\mathbf{x}_i \in \mathbb{R}^n$ and $y_i \in \{-1, +1\}$, SVM solves:

$$
\min_{\mathbf{w}, b, \boldsymbol{\xi}} \ \frac{1}{2} \|\mathbf{w}\|^2 + C \sum_{i=1}^n \xi_i
$$
$$
\text{subject to } \ y_i(\mathbf{w}^\top \mathbf{x}_i + b) \geq 1 - \xi_i, \quad \xi_i \geq 0
$$

Where:
- $\mathbf{w}$: weight vector (normal to the hyperplane)
- $b$: bias term
- $xi_i$: slack variables for misclassification
- $C > 0$: regularization parameter


##### Nonlinear SVM

To handle nonlinear boundaries, SVM maps data into a higher-dimensional space via a kernel function $K(\mathbf{x}_i, \mathbf{x}_j)$:

$$
K(\mathbf{x}_i, \mathbf{x}_j) = \phi(\mathbf{x}_i)^\top \phi(\mathbf{x}_j)
$$

Kernels:
- Linear: $K(\mathbf{x}, \mathbf{z}) = \mathbf{x}^\top \mathbf{z}$
- RBF: $K(\mathbf{x}, \mathbf{z}) = \exp(-\gamma \|\mathbf{x} - \mathbf{z}\|^2)$
- Polynomial: $K(\mathbf{x}, \mathbf{z}) = (\mathbf{x}^\top \mathbf{z} + r)^d$


The model:

- Has no distributional assumptions.
- Assumes classes are separable with margin.
- Is sensitive to feature scaling.


In [ ]:
# SVM Classifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

# ----------------------------------------- Model Parameters  -----------------------------------------

auto_best_threshold_based_on_metric = 'f1'      # Available metrics: ["accuracy", "sensitivity", "specificity", "ppv", "npv", "f1", "auc"]
include_bootstrap_confidence_intervals = True        # True # False
cross_validation_splits = 5                           # 5-Fold cross validation
grid_search_scoring_metric = 'recall'                 # choosing best metric in grid search : ['roc_auc','recall']

# ----------------------------------------------------------------------------------------------------

# scaling + SVM
svm_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(probability=True, kernel='rbf', class_weight='balanced', random_state=42))
])

# SVM hyperparameter grid
svm_param_grid = {
    'svc__C': [0.01, 0.1, 1, 10, 100],
    'svc__gamma': ['scale', 'auto'],
    'svc__kernel': ['rbf', 'linear', 'poly', 'sigmoid']
}

cv = StratifiedKFold(n_splits=cross_validation_splits, shuffle=True, random_state=random_state)
svm_grid = GridSearchCV(svm_pipe, svm_param_grid, cv=cv, scoring=grid_search_scoring_metric, n_jobs=-1) 
svm_grid.fit(X_train, y_train)

print("SVM best parameters:", svm_grid.best_params_)

# Test set evaluation
svm_best = svm_grid.best_estimator_
y_test_proba_svm = svm_best.predict_proba(X_test)[:, 1]

best_metrics_svm = find_best_threshold(
    model=svm_best,
    X_test=X_test,
    y_test=y_test,
    metric=auto_best_threshold_based_on_metric,             
    bootstrap=include_bootstrap_confidence_intervals,    # to get CI    
    n_bootstrap=100        
)

print_metrics(metrics_dict=best_metrics_svm)

plot_confusion_mat(svm_best, 
                   X_test, y_test, 
                   threshold=best_metrics_svm['threshold'], 
                   labels=["Human", "Robot"])

fig = plot_auc(y_test, y_test_proba_svm)
fig.show()
# plot_auc_paper(y_test, y_test_proba)

# ------------------------ Choosing a different threshold ------------------------

# threshold_to_display = 0.5 

# threshold_dict = evaluate_model_at_threshold(svm_best, 
#                             X_test, y_test, 
#                             threshold=threshold_to_display, 
#                             bootstrap=include_bootstrap_confidence_intervals, 
#                             n_bootstrap=100)

# print_metrics(metrics_dict=threshold_dict)

# plot_confusion_mat(svm_best, 
#                    X_test, y_test, 
#                    threshold=threshold_dict['threshold'], 
#                    labels=["Human", "Robot"])

# --------------------------------------------------------------------------------

### Decision Tree

A Decision Tree partitions the feature space into axis-aligned regions using recursive binary splits, aiming to maximize class purity.

#### Model Definition

Given data $(\mathbf{x}_i, y_i)$, a tree is built by selecting splits that maximize an impurity reduction criterion.

The impurity measure is the Gini impurity: $G = \sum_{k=1}^K p_k(1 - p_k)$  

Where $p_k$ is the proportion of class $k$ in a node.

#### Properties

- Interpretable and non-parametric.
- Handles both numerical and categorical data.
- Prone to overfitting without pruning.
- No need for feature scaling.

### Random Forest

Random Forest is an ensemble method that constructs multiple decision trees and combines their outputs via majority voting.

#### Model Definition

Given labeled data $(\mathbf{x}_i, y_i)$, where $\mathbf{x}_i \in \mathbb{R}^n$ and $y_i \in \{1, \dots, K\}$, the Random Forest predicts class $\hat{y}$ by:

$$
\hat{y} = \arg\max_{k \in \{1,\dots,K\}} \sum_{t=1}^T \mathbb{I}(h_t(\mathbf{x}) = k)
$$

Where:
- $h_t$: prediction of the $t$-th tree
- $T$: total number of trees

#### Properties

- Trained on bootstrap samples (with replacement).
- Splits use random feature subsets.
- No need for feature scaling.
- Captures nonlinearity and feature interactions.
- Reduces variance via averaging.

In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

# ----------------------------------------- Model Parameters  -----------------------------------------

auto_best_threshold_based_on_metric = 'accuracy'         # Available metrics: ["accuracy", "sensitivity", "specificity", "ppv", "npv", "f1", "auc"]
include_bootstrap_confidence_intervals = True           # True # False
cross_validation_splits = 5                              # 5-Fold cross validation
grid_search_scoring_metric = 'recall'                    # choosing best metric in grid search : ['roc_auc','recall']

# ----------------------------------------------------------------------------------------------------

rf_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestClassifier(random_state=random_state, class_weight='balanced'))
])

# Random Forest hyperparameter grid
rf_param_grid = {
    'rf__n_estimators': [100, 200],
    'rf__max_depth': [None, 5, 10],
    'rf__min_samples_split': [2, 5],
    'rf__min_samples_leaf': [1, 2]
}

cv = StratifiedKFold(n_splits=cross_validation_splits, shuffle=True, random_state=random_state)
rf_grid = GridSearchCV(rf_pipe, rf_param_grid, cv=cv, scoring=grid_search_scoring_metric, n_jobs=-1)
rf_grid.fit(X_train, y_train)

print("Random Forest best parameters:", rf_grid.best_params_)

# Evaluate RF on test set
rf_best = rf_grid.best_estimator_
y_test_proba_rf = rf_best.predict_proba(X_test)[:, 1]

best_metrics_rf = find_best_threshold(
    model=rf_best,
    X_test=X_test,
    y_test=y_test,
    metric=auto_best_threshold_based_on_metric,             
    bootstrap=include_bootstrap_confidence_intervals,    # to get CI    
    n_bootstrap=100        
)

print_metrics(metrics_dict=best_metrics_rf)

plot_confusion_mat(rf_best, 
                   X_test, y_test, 
                   threshold=best_metrics_rf['threshold'], 
                   labels=["Human", "Robot"])

fig = plot_auc(y_test, y_test_proba_rf)
fig.show()
# plot_auc_paper(y_test, y_test_proba_rf)

# #------------------------ Choosing a different threshold ------------------------

# threshold_to_display = 0.3 

# threshold_dict = evaluate_model_at_threshold(rf_best, 
#                             X_test, y_test, 
#                             threshold=threshold_to_display, 
#                             bootstrap=include_bootstrap_confidence_intervals, 
#                             n_bootstrap=100)

# print_metrics(metrics_dict=threshold_dict)

# plot_confusion_mat(rf_best, 
#                    X_test, y_test, 
#                    threshold=threshold_dict['threshold'], 
#                    labels=["Human", "Robot"])

# #--------------------------------------------------------------------------------

### Gradient Boosting Machine

Gradient Boosting builds an additive model by sequentially fitting decision trees to correct the errors of prior models using gradient descent on a loss function.

#### Model Definition

Given data $(\mathbf{x}_i, y_i)$, the prediction is:

$$
F_M(\mathbf{x}) = \sum_{m=1}^M \gamma_m h_m(\mathbf{x})
$$

Where:
- $h_m(\mathbf{x})$: the $m$-th weak learner (decision tree).
- $\gamma_m$: step size
- $M$: number of boosting rounds

Each $h_m$ is fit to the negative gradient of the loss function $\ell$:

$$
r_i^{(m)} = -\left[\frac{\partial \ell(y_i, F(\mathbf{x}_i))}{\partial F(\mathbf{x}_i)}\right]_{F = F_{m-1}}
$$

In [ ]:
# BGM
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.utils.class_weight import compute_sample_weight

# ----------------------------------------- Model Parameters  -----------------------------------------

auto_best_threshold_based_on_metric = 'accuracy'       # Available metrics: ["accuracy", "sensitivity", "specificity", "ppv", "npv", "f1", "auc"]
include_bootstrap_confidence_intervals = True          # True # False
cross_validation_splits = 5                            # 5-Fold cross validation
grid_search_scoring_metric = 'recall'                 # choosing best metric in grid search : ['roc_auc','recall']

# ----------------------------------------------------------------------------------------------------

gb_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('gb', GradientBoostingClassifier(random_state=42))
])

# Gradient Boosting hyperparameter grid
gb_param_grid = {
    'gb__n_estimators': [50, 100, 200, 300, 400, 500],
    'gb__learning_rate': [0.01, 0.1, 0.2, 0.3, 0.4],
    'gb__max_depth': [2, 3, 5, 7]
}

sample_weight = compute_sample_weight(class_weight='balanced', y=y_train)
cv = StratifiedKFold(n_splits=cross_validation_splits, shuffle=True, random_state=random_state)
gb_grid = GridSearchCV(gb_pipe, gb_param_grid, cv=cv, scoring=grid_search_scoring_metric, n_jobs=-1)
gb_grid.fit(X_train, y_train, gb__sample_weight=sample_weight)

print("Gradient Boosting best parameters:", gb_grid.best_params_)

# Evaluate GB on test set
gb_best = gb_grid.best_estimator_
y_test_proba_gb = gb_best.predict_proba(X_test)[:, 1]


best_metrics_gb = find_best_threshold(
    model=gb_best,
    X_test=X_test,
    y_test=y_test,
    metric=auto_best_threshold_based_on_metric,             
    bootstrap=include_bootstrap_confidence_intervals,    # to get CI    
    n_bootstrap=100        
)

print_metrics(metrics_dict=best_metrics_gb)

plot_confusion_mat(gb_best, 
                   X_test, y_test, 
                   threshold=best_metrics_gb['threshold'], 
                   labels=["Human", "Robot"])

fig = plot_auc(y_test, y_test_proba_gb)
fig.show()
# plot_auc_paper(y_test, y_test_proba_gb)

# ------------------------ Choosing a different threshold ------------------------

# threshold_to_display = 0.5 

# threshold_dict = evaluate_model_at_threshold(gb_best, 
#                             X_test, y_test, 
#                             threshold=threshold_to_display, 
#                             bootstrap=include_bootstrap_confidence_intervals, 
#                             n_bootstrap=100)

# print_metrics(metrics_dict=threshold_dict)

# plot_confusion_mat(gb_best, 
#                    X_test, y_test, 
#                    threshold=threshold_dict['threshold'], 
#                    labels=["Human", "Robot"])

# --------------------------------------------------------------------------------

### k-Nearest Neighbors (KNN)

k-NN is a non-parametric, instance-based learning algorithm that predicts the output of a query point based on the outputs of its *k* nearest neighbors in the training set.

#### Model Definition

Given a query point $\mathbf{x}$ and a training set $\{(\mathbf{x}_i, y_i)\}_{i=1}^N$, the prediction $\hat{y}$ is:

- **For regression**:
$$
\hat{y} = \frac{1}{k} \sum_{i \in \mathcal{N}_k(\mathbf{x})} y_i
$$

- **For classification**:
$$
\hat{y} = \text{mode}\left(\{y_i \mid i \in \mathcal{N}_k(\mathbf{x})\}\right)
$$

Where:
- $\mathcal{N}_k(\mathbf{x})$: indices of the $k$ nearest neighbors to $\mathbf{x}$.

No explicit training is performed; computation occurs at prediction time.


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# ----------------------------------------- Model Parameters  -----------------------------------------

auto_best_threshold_based_on_metric = 'accuracy'      # Available metrics: ["accuracy", "sensitivity", "specificity", "ppv", "npv", "f1", "auc"]
include_bootstrap_confidence_intervals = True        # True # False
cross_validation_splits = 5                           # 5-Fold cross validation
grid_search_scoring_metric = 'roc_auc'                 # choosing best metric in grid search : ['roc_auc','recall']

# ----------------------------------------------------------------------------------------------------

knn_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier())
])

# KNN hyperparameter grid
knn_param_grid = {
    'knn__n_neighbors': [3, 5, 7, 9, 11],
    'knn__weights': ['uniform', 'distance'],
    'knn__metric': ['euclidean', 'manhattan']
}

cv = StratifiedKFold(n_splits=cross_validation_splits, shuffle=True, random_state=random_state)
knn_grid = GridSearchCV(knn_pipe, knn_param_grid, cv=cv, scoring=grid_search_scoring_metric, n_jobs=-1)
knn_grid.fit(X_train, y_train)

print("KNN best parameters:", knn_grid.best_params_)

# Evaluate KNN on test set
knn_best = knn_grid.best_estimator_
y_test_proba_knn = knn_best.predict_proba(X_test)[:, 1]

best_metrics_knn = find_best_threshold(
    model=knn_best,
    X_test=X_test,
    y_test=y_test,
    metric=auto_best_threshold_based_on_metric,
    bootstrap=include_bootstrap_confidence_intervals,
    n_bootstrap=100
)

print_metrics(metrics_dict=best_metrics_knn)

plot_confusion_mat(knn_best,
                   X_test, y_test,
                   threshold=best_metrics_knn['threshold'],
                   labels=["Human", "Robot"])

fig = plot_auc(y_test, y_test_proba_knn)
fig.show()
# plot_auc_paper(y_test, y_test_proba_knn)

# ------------------------ Choosing a different threshold ------------------------

# threshold_to_display = 0.08

# threshold_dict = evaluate_model_at_threshold(knn_best, 
#                             X_test, y_test, 
#                             threshold=threshold_to_display, 
#                             bootstrap=include_bootstrap_confidence_intervals, 
#                             n_bootstrap=100)

# print_metrics(metrics_dict=threshold_dict)

# plot_confusion_mat(knn_best, 
#                    X_test, y_test, 
#                    threshold=threshold_dict['threshold'], 
#                    labels=["Human", "Robot"])

# --------------------------------------------------------------------------------

In [ ]:
# Comparison between the models

fig_radar = radar_metrics_multi(
    [best_metrics_logreg, best_metrics_svm, best_metrics_rf, best_metrics_gb, best_metrics_knn],
    ["Logistic Regression", "SVM", "Random Forest", "GBM", "KNN"])
fig_radar.show()

# fig = radar_metrics_multi_matplotlib(
#     [best_metrics_logreg, best_metrics_svm, best_metrics_rf, best_metrics_gb, best_metrics_knn],
#     ["Logistic Regression", "SVM", "Random Forest", "GBM", "KNN"])
# fig.show()

In [ ]:
# Feature Importance

import matplotlib.pyplot as plt
import numpy as np

# Prepare feature names and importance values
features = X_train.columns
logreg_vals = abs(best_model.coef_[0])
rf_vals = rf_best.named_steps['rf'].feature_importances_
gb_vals = gb_best.named_steps['gb'].feature_importances_

# Sort by importance
def sort_features(features, importances):
    sorted_idx = np.argsort(importances)[::-1]
    return np.array(features)[sorted_idx], np.array(importances)[sorted_idx]

logreg_features, logreg_vals = sort_features(features, logreg_vals)
rf_features, rf_vals = sort_features(features, rf_vals)
gb_features, gb_vals = sort_features(features, gb_vals)

# Plot
fig, axes = plt.subplots(1, 3, figsize=(12, 3), sharey=True)

bars0 = axes[0].barh(logreg_features, logreg_vals, color='skyblue')
axes[0].set_title("Logistic Regression\n(Absolute Coefficients)")
axes[0].invert_yaxis()
axes[0].bar_label(bars0, fmt='%.3f')

bars1 = axes[1].barh(rf_features, rf_vals, color='lightgreen')
axes[1].set_title("Random Forest\n(Mean Decrease in Impurity)")
axes[1].bar_label(bars1, fmt='%.3f')

bars2 = axes[2].barh(gb_features, gb_vals, color='salmon')
axes[2].set_title("Gradient Boosting\n(Mean Decrease in Impurity)")
axes[2].bar_label(bars2, fmt='%.3f')

plt.tight_layout()
plt.show()
